In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
urls = [
    "https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_White",
    "https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Blue",
    "https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Black",
    "https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Red",
    "https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Green",
]

In [3]:
jumpstart_decklist_df = pd.DataFrame()

In [4]:
for url in urls:
    print(f"Scrap url: '{url}'")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    decktitle_divs = soup.find_all("div", {"class": "ext-scryfall-decktitlecontainer"})
    deckcontents_divs = soup.find_all("div", {"class": "ext-scryfall-deckcontents"})
    assert len(decktitle_divs) == len(deckcontents_divs)

    for decktitle_div, deckcontents_div in zip(decktitle_divs, deckcontents_divs):
        deck_name = decktitle_div.find("span", {"class": "ext-scryfall-decktitle"}).text
        decklist_ps = deckcontents_div.find_all("p", {"class": "ext-scryfall-deckentry"})
        for decklist_p in decklist_ps:
            card_name = decklist_p.find("a", {"class": "ext-scryfall-cardname"}).text
            card_count = decklist_p.find("span", {"class": "ext-scryfall-deckcardcount"}).text
            
            jumpstart_decklist_df = pd.concat([
                jumpstart_decklist_df,
                pd.DataFrame([{
                    "deck_name": deck_name,
                    "card_name": card_name,
                    "card_count": card_count
                }])
            ])

Scrap url: 'https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_White'
Scrap url: 'https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Blue'
Scrap url: 'https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Black'
Scrap url: 'https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Red'
Scrap url: 'https://mtg.fandom.com/wiki/Jumpstart/Decklists_-_Green'


In [5]:
jumpstart_decklist_df = jumpstart_decklist_df.reset_index(drop=True)

In [6]:
jumpstart_decklist_df

,deck_name,card_name,card_count
0,Basri,Basri's Acolyte,1
1,Basri,Basri's Lieutenant,1
2,Basri,Concordia Pegasus,1
3,Basri,Knight of the Tusk,1
4,Basri,Siege Striker,1
...,...,...,...
1661,Tree-Hugging 4,Explore,1
1662,Tree-Hugging 4,Nature's Way,1
1663,Tree-Hugging 4,Verdant Embrace,1
1664,Tree-Hugging 4,Thriving Grove,1


In [7]:
jumpstart_decklist_df.to_csv("../data/jmp_decklist.csv", index=False)